# DS 3000 Quiz 4

Due by: Tuesday Dec 6 @ 11:59 PM EST

Time Limit: You have 2 hours to complete the assignment once started

## Instructions

This quiz has 100 points total.

- You are welcome to post a private note on piazza, but to keep a consistent testing environment for all students we are unlikely to provide assistance.
- You may not contact other students with information about this this quiz
    - even saying "it was easy/hard" in a general sense can introduce a bias in favor of students who take the quiz earlier or later
- Under no circumstances should you share a copy of this quiz with anyone who isn't a member of the course staff.
- Take this quiz with open notes and feel free to access any online resource / documentation you'd like.  

### Submission Instructions
After completing the quiz below, please follow the instructions below to submit:
1. "Kernel" -> "Restart & Run All"
1. save your quiz file to this latest version
1. upload the `.ipynb` to gradescope **before** clicking submit
1. ensure that you can see your jupyter notebook in the gradescope interface after clicking "submit"

We specify the last note above as gradescope has allowed students to "submit" without uploading a file.  It is your responsibility to ensure that you've actually submitted a file.

### Academic Integrity Pledge

Input your name below to sign the Academic Integrity Pledge before continuing with the quiz. Failure to do so will result in a score of **0**.

In [1]:
name = 'Student Name Here'
print(f'I, {name}, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source other than private messages between myself and the professor on Piazza/via email.')

I, Student Name Here, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source other than private messages between myself and the professor on Piazza/via email.


In [2]:
# the following modules will be necessary to complete the quiz
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
from copy import copy
from collections import Counter
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pylab as py
import scipy.stats as stats
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from pca import pca

### Part 1: Regression (30 points; 10 points each)

For this first part, you will be using the `savings.csv` data set which is available in the Quiz Module on Canvas. These data consist of 50 observations of 5 variables, measuring for 50 different countries in the 1960s:
- sr: aggregate personal savings
- pop15: % of population under 15
- pop75: % of population over 75
- dpi: real per-capita disposable income
- ddpi: % growth rate of dp

We will attempt to build a regression model that can help predict aggregate personal savings.

In [3]:
df_save = pd.read_csv('savings.csv', index_col=0)
df_save.head()

,sr,pop15,pop75,dpi,ddpi
Australia,11.43,29.35,2.87,2329.68,2.87
Austria,12.07,23.32,4.41,1507.99,3.93
Belgium,13.17,23.80,4.43,2108.47,3.82
Bolivia,5.75,41.89,1.67,189.13,0.22
Brazil,12.88,42.19,0.83,728.47,4.56


### Part 1.1: Cross Validation and $R^2$

Conduct a 10-fold Cross Validation Multiple Regression, using `pop15`, `pop75`, `dpi`, and `ddpi` to predict `sr`. Calculate the cross validated $R^2$. In a markdown cell, (a) interpret $R^2$ in the context of the problem and (b) discuss if it seems like a good value to you.

### Part 1.2: Fitting Full Model and Interpreting Line

Now fit the full data set to the multiple regression line. Use the function provided below to print out the estimated multiple regression line (you should call it with something like `print_sr_line(reg.intercept_, reg.coef_)`. Then, **in a markdown cell**, interpret (a) the intercept and (b) the coefficient for the `pop75` x-feature.

In [4]:
def print_sr_line(intercept, coef):
    intercept = intercept.round(2)
    coef = coef.round(4)
    print(f'sr = {intercept} + {coef[0]} * pop15 + {coef[1]} * pop75 + {coef[2]} * dpi + {coef[3]} * ddpi')

### Part 1.3: Checking Assumptions and Making Recommendations

Create 6 error plots to assess whether we can trust this model to predict aggregate personal savings. You should plot:
- The errors across the index
- The errors across each x-feature
    - population under 15
    - population over 75
    - disposable income
    - growth in disposable income
- The normal probability plot of the errors

Then, **in a markdown cell**, write a paragraph summarizing whether the model meets all the assumptions for using multiple linear regression; if so, do you recommend we use the model and if not, why not?

## Part 2: K-Means (30 points; 10 points each part)



For this problem we will see if we can identify any patterns in some weather and pollution data taken every day in Chicago from January 1st 2000 to December 31st 2005. You will need the `pollution.csv` data set from the Quiz Module on Canvas. We will conduct K-Means using the numeric features:
- tmpd (temperature in Fahrenheit)
- dptp (dewpoint in Fahrenheit)
- pm10tmean2 (particulate matter)
- o3tmean2 (oxygen)
- no2tmean2 (nitrogen dioxide)

In [5]:
df_chic = pd.read_csv('pollution.csv')
df_chic.dropna(inplace=True)
df_chic

,city,tmpd,dptp,date,pm10tmean2,o3tmean2,no2tmean2
0,chic,42.0,32.2,2000-01-01,29.333333,4.621212,24.069444
1,chic,48.0,42.3,2000-01-02,47.500000,16.527778,16.312500
2,chic,35.0,33.7,2000-01-03,11.500000,9.099034,15.114583
3,chic,28.0,22.2,2000-01-04,19.000000,11.059179,23.801786
4,chic,20.0,16.6,2000-01-05,23.500000,4.352657,20.349802
...,...,...,...,...,...,...,...
2187,chic,40.0,33.6,2005-12-27,27.000000,4.468750,23.500000
2188,chic,37.0,34.5,2005-12-28,27.500000,3.260417,19.285628
2189,chic,35.0,29.4,2005-12-29,23.500000,6.794837,19.972222
2190,chic,36.0,31.0,2005-12-30,19.200000,3.034420,22.805556


### Part 2.1: Prepare the Data and Choose K

Scale the numeric features using scale normalization, and then conduct K-Means using choices of $k$ from 2 to 20. Keep track of the mean distance squared to the centroids under each choice of $k$, then plot them. **In a markdown cell**, make a choice of $k$ and explain why you made that choice.

### Part 2.2: Run K-Means

Regardless of your own choice of $k$ from the previous part, run K-Means with $k=5$ and save the cluster assignments as a column in the data set. Print the head of this data set to show that this worked.

### Part 2.3: Make Inferences and Discuss

Instead of looking at the raw data set to make some conclusions, we will make two scatter plots using plotly.express:
- Make a plot of date (x-axis) vs. tmpd (y-axis) and color by cluster
- Make a plot of date (x-axis) vs. pm10tmean2 (y-axis) and color by cluster

Then, **in a markdown cell** discuss in a paragraph what the two plots you've created tell you about how the K-Means algorithm has clustered the data and what the five clusters seem to represent.

## Part 3: Principal Component Analysis 1 (20 points; 10 points each part)

For this part we will use the `leafshape.csv` data set available in the Quiz Module on Canvas. The data set contains leaf length, width and petiole measurements taken at various sites worldwide. The features in the data are:

- bladelen: leaf length (in mm)
- petiole: leaf petiole
- bladewid: leaf width (in mm)
- latitude: latitude of the plant
- logwid: natural log of width
- logpet: natural log of petiole
- loglen: natural log of length
- arch: leaf architecture (0 = plagiotropic, 1 = orthotropic)
- location: a factor with 6 levels corresponding to location

### Part 3.1: PCA "map"/biplot

Conduct PCA on the first seven features (all except `arch` and `location`). Store the first two principal components as columns in the data set. Then make three "map"/biplots:
- One using plotly.express and the `px.scatter` function, which colors the points by `location`.
    - Save this file as `pca_leaf_biplot_loc.html` **and submit it with your Quiz to Gradescope**
- One using plotly.express and the `px.scatter` function, which colors the points by `arch`.
- One using `.biplot()` function from the `pca` module
    - You will need to make sure the `pca` module is installed and to re-run PCA using this module.

In [6]:
df_leaf = pd.read_csv('leafshape.csv')
df_leaf.head()

,bladelen,petiole,bladewid,latitude,logwid,logpet,loglen,arch,location
0,33.88,1.402632,13.65,5.0,2.613740,0.338350,3.522825,0,Sabah
1,33.32,1.016260,10.26,5.0,2.328253,0.016129,3.506158,0,Sabah
2,29.35,2.392025,12.21,5.0,2.502255,0.872140,3.379293,0,Sabah
3,26.87,0.808787,8.70,5.0,2.163323,-0.212220,3.291010,0,Sabah
4,26.67,0.802767,8.41,5.0,2.129421,-0.219691,3.283539,0,Sabah


### Part 3.2: Discussion

In a markdown cell, write a paragraph discussing what the "map"/biplot tells you about:
- how the features are related
- if there are any groupings/clusters in the data
- which features are considered most important within the first two principal components

## Part 4: Principal Component Analysis 2 (20 points; 10 points each part)

Investors often use commodity prices to predict overall market sentiment (sometimes defined as a categorical feature: bearish or bullish). The `commodity_prices.csv` file in the Quiz folder on Canvas includes 30 commodotiy prices over time. We will see if we can use Principal Component Analysis to reduce the feature set without too much loss of information in preparation for predicting market sentiment with a supervised learning algorithm (we will not do this part, but you should keep in mind that this would be the next step).

### Part 4.1: Dimension Reduction

Implement the PCA algorithm on all features in the data except `Year` (so, using all the commodities). Then:
- create a scree line plot (**not** a bar plot; you will need to slightly adapt the bar plot code from lecture) of the explained variance ratio of each principal component
- calculate the cumulative explained variance ratio at each principal component

In [7]:
df_price = pd.read_csv('commodity_prices.csv')
df_price.dropna(inplace=True)
df_price.head()

,Year,Cocoa,Coffee,Tea,Crude Oil,Coal,Natural Gas,Banana,Sugar,Orange,...,Silver,Cotton,Rubber,Tobacco,Coconut Oil,Groundnut Oil,Palm Oil,Soybean,Logs,Sawnwood
11,1971,0.54,0.99,0.85,1.69,8.68,0.18,0.14,0.10,0.15,...,1.55,0.78,0.33,1042.65,365.53,440.62,260.94,303.75,43.27,175.72
12,1972,0.64,1.11,0.78,1.82,9.60,0.19,0.14,0.16,0.15,...,1.68,0.83,0.33,1073.64,234.06,425.93,217.26,240.58,41.12,170.93
13,1973,1.13,1.37,0.79,2.81,11.48,0.21,0.15,0.21,0.16,...,2.56,1.39,0.68,1128.02,513.00,546.17,377.54,436.00,68.03,223.53
14,1974,1.56,1.45,1.09,10.97,16.89,0.29,0.18,0.65,0.18,...,4.71,1.46,0.75,1482.07,997.58,1076.89,669.05,832.17,81.66,246.83
15,1975,1.25,1.44,1.14,10.43,26.84,0.43,0.22,0.45,0.23,...,4.42,1.22,0.56,1841.99,393.50,857.50,434.17,563.33,67.51,223.44


### Part 4.2: Discussion

An investment firm has ask you to build a model to predict market sentiment based on these commodity prices. In a markdown cell, write 2-3 paragraphs as if you were discussing what you've found with the PCA and how this might help you move forward with predicting market sentiment. You should be sure to mention:
- Why you would implement PCA in the first place
- How many components you would suggest are useful to keep
    - One of the investors who has done some PCA before suggests that 99% is a good benchmark to use in such a problem
- What sort of supervised learning algorithm would you use to predict the binary classes (bearish/bullish) of overall market sentiment